# 데이터프레임 전처리

In [13]:
import json
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [14]:
# 데이터 불러오기
MP = pd.read_json('MP.json', orient = 'table')
PB = pd.read_json("PB.json", orient = 'table')

In [15]:
# 국회의원 별 df
MP

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
0,강기윤,국민의힘,[보건복지위원회],경남 창원시성산구,남,재선,지역구
1,강대식,국민의힘,"[국토교통위원회, 예산결산특별위원회]",대구 동구을,남,초선,지역구
2,강득구,더불어민주당,"[교육위원회, 국회운영위원회, 예산결산특별위원회]",경기 안양시만안구,남,초선,지역구
3,강민국,국민의힘,"[민생경제안정특별위원회, 정무위원회, 예산결산특별위원회]",경남 진주시을,남,초선,지역구
4,강민정,더불어민주당,"[중앙선거관리위원회위원(남래진)선출에관한인사청문특별위원회, 교육위원회, 국회운영...",비례대표,여,초선,비례대표
...,...,...,...,...,...,...,...
294,홍익표,더불어민주당,[문화체육관광위원회],서울 중구성동구갑,남,3선,지역구
295,홍정민,더불어민주당,"[산업통상자원중소벤처기업위원회, 여성가족위원회]",경기 고양시병,여,초선,지역구
296,황보승희,국민의힘,[문화체육관광위원회],부산 중구영도구,여,초선,지역구
297,황운하,더불어민주당,[정무위원회],대전 중구,남,초선,지역구


In [16]:
# 법률안 별 df
PB.head()

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
0,성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김상훈, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, 조수진, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1,남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김승수, 배준영, 유경준, 유의동, 이명수, 정희용, 조수진, 지성호]",환경노동위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
2,양성평등기본법 | 일부개정법률안,강대식,"[김병욱, 김상훈, 김승수, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
3,여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 | 일부개정법률안,소병철,"[김승남, 김정호, 김회재, 서동용, 이수진, 이용선, 정일영, 주철현, 한준호]",행정안전위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
4,조세특례제한법 | 일부개정법률안,정우택,"[김학용, 서정숙, 이양수, 이종성, 정운천, 정희용, 조명희, 지성호, 최승재, ...",기획재정위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


In [17]:
# 의원별로 새로운 df 생성
person_li = MP['의원명'].tolist() + MP['의원명'].tolist()

In [20]:
len(person_li), len(set(person_li))

(598, 297)

In [28]:
person_se = pd.Series(person_li)
name_pb = pd.DataFrame(person_se, columns = ['의원명'])

In [29]:
name_pb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의원명     598 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [30]:
#이름 오름차순 정렬
name_pb = name_pb.sort_values(['의원명'])
#인덱스 다시 설정
name_pb.reset_index()
# name_pb = name_pb.drop('index', axis = 1)

,index,의원명
0,0,강기윤
1,299,강기윤
2,300,강대식
3,1,강대식
4,2,강득구
...,...,...
593,296,황보승희
594,596,황운하
595,297,황운하
596,298,황희


In [31]:
name_pb.reset_index(inplace=True, drop=True)

In [32]:
name_pb

,의원명
0,강기윤
1,강기윤
2,강대식
3,강대식
4,강득구
...,...
593,황보승희
594,황운하
595,황운하
596,황희


In [33]:
type(name_pb)

pandas.core.frame.DataFrame

In [34]:
# 발의자 설정(대표/ 공동)
name_pb['발의자'] = '대표'
name_pb.iloc[1::2,1] = '공동'

In [35]:
name_pb

,의원명,발의자
0,강기윤,대표
1,강기윤,공동
2,강대식,대표
3,강대식,공동
4,강득구,대표
...,...,...
593,황보승희,공동
594,황운하,대표
595,황운하,공동
596,황희,대표


In [36]:
# 법률안명 추가
name_pb['법률안명'] = ""
name_pb['소관위원'] = ''

In [37]:
name_pb

,의원명,발의자,법률안명,소관위원
0,강기윤,대표,,
1,강기윤,공동,,
2,강대식,대표,,
3,강대식,공동,,
4,강득구,대표,,
...,...,...,...,...
593,황보승희,공동,,
594,황운하,대표,,
595,황운하,공동,,
596,황희,대표,,


In [39]:
PB

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
0,성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김상훈, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, 조수진, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1,남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김승수, 배준영, 유경준, 유의동, 이명수, 정희용, 조수진, 지성호]",환경노동위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
2,양성평등기본법 | 일부개정법률안,강대식,"[김병욱, 김상훈, 김승수, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
3,여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 | 일부개정법률안,소병철,"[김승남, 김정호, 김회재, 서동용, 이수진, 이용선, 정일영, 주철현, 한준호]",행정안전위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
4,조세특례제한법 | 일부개정법률안,정우택,"[김학용, 서정숙, 이양수, 이종성, 정운천, 정희용, 조명희, 지성호, 최승재, ...",기획재정위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
...,...,...,...,...,...,...,...
1761,환경영향평가법 | 일부개정법률안,태영호,"[김영식, 김정재, 박성민, 박진, 배준영, 유경준, 이종배, 이주환, 이채익, 조명희]",환경노동위원회,2022-01-03,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1762,전기통신사업법 | 일부개정법률안,윤영찬,"[강병원, 박광온, 서삼석, 설훈, 양정숙, 오영환, 이용빈, 이학영, 최혜영]",과학기술정보방송통신위원회,2022-01-03,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1763,산업기술의 유출방지 및 보호에 관한 법률 | 일부개정법률안,김경만,"[강득구, 강선우, 김정호, 송기헌, 송옥주, 이동주, 이수진, 이용빈, 임종성, ...",산업통상자원중소벤처기업위원회,2022-01-03,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1764,온라인 국민참여포털 국민신문고의 설치 및 운영에 관한 | 법률안,이정문,"[강선우, 강준현, 김경만, 김병기, 김상희, 김영진, 김종민, 김철민, 김한정, ...",정무위원회,2022-01-03,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


In [9]:
for i in name_pb['의원명'].tolist():
    for t in range(len(list(PB['대표발의자']))):
        if i == PB['대표발의자'][t]:
            name_pb['법률안명'][(name_pb['발의자']=='대표')&(name_pb['의원명']==i)] = name_pb['법률안명'][(name_pb['발의자']=='대표')&(name_pb['의원명']==i)] + ',' + PB['법률안명'][i == PB['대표발의자']]
name_pb

,의원명,발의자,법률안명,소관위원
0,강기윤,대표,NaN,
299,강기윤,공동,,
300,강대식,대표,NaN,
1,강대식,공동,,
2,강득구,대표,NaN,
...,...,...,...,...
296,황보승희,공동,,
596,황운하,대표,NaN,
297,황운하,공동,,
298,황희,대표,,


In [10]:
name_pb['법률안명'][(name_pb['발의자']=='대표')&(name_pb['의원명']=='황운하')]

596    NaN
Name: 법률안명, dtype: object

In [11]:
name_pb['법률안명'][(name_pb['발의자']=='대표')&(name_pb['의원명']=='황운하')] + ',' + PB['법률안명']['황운하' == PB['대표발의자']]

596     NaN
872     NaN
1183    NaN
1202    NaN
1449    NaN
Name: 법률안명, dtype: object

In [12]:
PB['법률안명']['황운하' == PB['대표발의자']]

872     국가공무원법 | 일부개정법률안
1183      변호사법 | 일부개정법률안
1202       광업법 | 일부개정법률안
1449     공직선거법 | 일부개정법률안
Name: 법률안명, dtype: object